This is the pipeline that is being run every week

In [1]:
# Select the week
WEEK_NUMBER = 3

## Load global settings

In [2]:
# Package imports
import json
import yaml
import pandas as pd
from tqdm import tqdm

# Local imports
from scripts.utils import read_files_to_dict


with open("settings.yaml") as f:
    settings = yaml.safe_load(f)


# Weeks
WEEK = settings["weeks"][WEEK_NUMBER]

# Global canvas settings
COURSE_ID = settings["global"]["canvas"]["course_id"]
ASSIGNMENT_ID = WEEK["canvas"]["assignment_id"]
QUIZ_ID = WEEK["canvas"]["quiz_id"]
R_QUIZ_QUESTION_ID = WEEK["canvas"]["r_quiz_question_id"]
ADV_QUIZ_QUESTION_ID = WEEK["canvas"]["adv_quiz_question_id"]

# Paths
RESOURCES_PATH = settings["global"]["paths"]["resources"]
SUBMISSIONS_PATH = settings["global"]["paths"]["submissions"]
STUDENT_SUBMISSION_TEMPLATE = settings["global"]["paths"]["student_submission_template"]
STUDENT_SUBMISSION_JSON_TEMPLATE = settings["global"]["paths"]["student_submission_json_template"]
LLM_COMPLETION_REPORT_TEMPLATE = settings["global"]["paths"]["llm_completion_report_template"]
LLM_FEEDBACK_REPORT_TEMPLATE = settings["global"]["paths"]["llm_feedback_report_template"]
LLM_GRADING_REPORT_TEMPLATE = settings["global"]["paths"]["llm_grading_report_template"]

# LLM Settings
MODEL = settings["global"]["llm"]["model"]
GRADING_TEMPERATURE = settings["global"]["llm"]["grading_temperature"]
FEEDBACK_TEMPERATURE = settings["global"]["llm"]["feedback_temperature"]
N_CHOICES_GRADING = settings["global"]["llm"]["n_choices_grading"]
N_CHOICES_FEEDBACK = settings["global"]["llm"]["n_choices_feedback"]
PROMPTS = {k: read_files_to_dict(settings["global"]["llm"]["prompts"][k]) for k in settings["global"]["llm"]["prompts"].keys()}
GLOBAL_RUBRICS = WEEK["global_rubrics"]
LOCK_GRADES_DATE = WEEK["lock_grades_date"]


## Get student submissions

The most recent valid submission is downloaded for each student and jsonified.

In [3]:
# Package imports
from canvasapi import Canvas
from canvasapi.requester import Requester
from canvas_connector.utils.canvas_utils import get_assignment_submissions_with_history, whitelist_submissions, assemble_canvas_file_submissions, get_most_recent_valid_submissions
import os
import zipfile
import shutil
import re

# Local imports
from scripts.filehandling import make_student_folders
from scripts.jsonify import jsonify
from scripts.utils import create_file_list


# Get environment variables
CANVAS_API_URL = os.getenv("CANVAS_API_URL")
CANVAS_API_KEY = os.getenv("CANVAS_API_KEY")

# Prepare connection to Canvas API
canvas = Canvas(CANVAS_API_URL, CANVAS_API_KEY)
requester = Requester(CANVAS_API_URL, CANVAS_API_KEY)

In [5]:
# TODO: Sort students by group and first do all A Group Students, then B Group Students?
# TODO: Separate grading report from feedback report, first create grading for all, then feedback for A, then upload, then feedback for B
def deduplicate_files_with_manual_fixes(file_list):

    # Helper function to extract que and att identifiers from a filename
    def extract_identifiers(filename):
        match = re.search(r'que-(\d+)_att-(\d+)', filename)
        if match:
            return match.group(1), match.group(2)
        return None, None

    # Dictionary to hold the preferred file for each identifier pair
    preferred_files = {}

    for file in file_list:
        que, att = extract_identifiers(file)
        if (que, att) == (None, None):
            continue  # Skip files that don't match the pattern

        if (que, att) not in preferred_files or "ManualFixes" in file:
            # Update if no file is recorded yet or if this file has "ManualFixes"
            preferred_files[(que, att)] = file

    # Return the list of preferred files
    return list(preferred_files.values())

In [ ]:
# Get all student submissions 
all_submissions = get_assignment_submissions_with_history(requester, COURSE_ID, ASSIGNMENT_ID)
student_ids = list({submission.user_id for submission in all_submissions})

# Loop student_ids
for student_id in student_ids:
    
    # Submissions of the current student
    student_submissions = whitelist_submissions(all_submissions, [student_id])
    if len(student_submissions) == 0:
        print(f"Student {student_id} has no submissions")
        # TODO: Add log 
        continue

    # Get canvas file submissions
    file_submissions = assemble_canvas_file_submissions(student_submissions)

    # Get all files of most recent (valid) submission
    valid_submissions = get_most_recent_valid_submissions(file_submissions)
    if len(valid_submissions) == 0:
        print(f"Student {student_id} has no valid submissions") # TODO add log
        continue
    
    # Download valid submissions
    for submission in valid_submissions:
        out_path = STUDENT_SUBMISSION_TEMPLATE.format(
            student_id=student_id,
            week=WEEK_NUMBER,
            assignment_id=ASSIGNMENT_ID,
            attempt=submission.attempt,
            question_id=submission.question_id,
            attachment_id=submission.attachment_id,
            file_extension=submission.file_extension
        )
        # check if file already exists
        submission.download(out_path)
        out_path = out_path + submission.file_extension
        if ".zip" in out_path:
            with zipfile.ZipFile(out_path) as zip_file:
                for member in zip_file.namelist():
                    filename = os.path.basename(member)
                    # skip directories
                    if not filename:
                        continue
                    if filename.startswith("._"):
                        continue
                    if ".DS_Store" in filename:
                        continue
                
                    # copy file (taken from zipfile's extract)
                    source = zip_file.open(member)
                    target = open(out_path.replace(submission.file_extension, f"_{filename}"), "wb")
                    with source, target:
                        shutil.copyfileobj(source, target)
            

        # Jsonify valid submissions
        submission_files = create_file_list("submissions", [f"stu-{student_id}", f"ass-{ASSIGNMENT_ID}", f"try-{submission.attempt}"], indicators_neg=[".json", ".pdf", ".md", ".zip", ".png"])
        submission_files = deduplicate_files_with_manual_fixes(submission_files)
        out_path = STUDENT_SUBMISSION_JSON_TEMPLATE.format(
            student_id=student_id, 
            week=WEEK_NUMBER,
            assignment_id=ASSIGNMENT_ID,
            attempt=submission.attempt
        )
        jsonify(submission_files, out_path)    

### Check if all students have submissions

In [ ]:
import re

from scripts.utils import load_jsonified_resources

def count_question_indicators_per_group(indicators: list, groups=[r"#R(\d+)", r"#Radv(\d+)", r"#Python(\d+)"]):
    counts = {group.replace("(\d+)", ""): len(re.compile(group).findall(", ".join(indicators))) for group in groups}
    return counts
student_ids = [int(folder.replace("stu-", "")) for folder in os.listdir(SUBMISSIONS_PATH) if folder.startswith("stu-")]


jsonified_resources = load_jsonified_resources(WEEK_NUMBER) # TODO: Check all keys and log if it is incorrect



desired_counts = count_question_indicators_per_group(list(jsonified_resources["questions"].keys()))
desired_counts_keys = {f"{k}_{v}": [] for k,v in desired_counts.items()}
completed_keys = {"R_completed": [], "Radv_completed": [], "Python_completed": []}
quality_check_dat = pd.DataFrame({"student_id": [], **desired_counts_keys, "submission_found": [], **completed_keys})

submission_files = create_file_list(SUBMISSIONS_PATH, ["submission.json", str(ASSIGNMENT_ID)])
for student_id in student_ids:
    student_found = False
    for file in submission_files:
        file_student_id = int(re.compile(r"stu-(\d+)").search(file).group(1))
        if file_student_id != student_id:
            continue
        submission_json = json.load(open(file))
        counts = count_question_indicators_per_group(list(submission_json.keys()))
        counts_row = {k: [v] for k, v in zip(desired_counts_keys.keys(), counts.values())}
        completed = {k: [c == dc] for k, c, dc in zip(completed_keys.keys(), counts.values(), desired_counts.values())}
        row = pd.DataFrame({"student_id": [student_id], **counts_row, "submission_found": [1], **completed})
        quality_check_dat = pd.concat([quality_check_dat, row])
        student_found = True
        break
    if not student_found:
        row = pd.DataFrame({"student_id": student_id, **{k: [0] for k in desired_counts_keys.keys()}, "submission_found": [0], "R_completed": [0], "Radv_completed": [0], "Python_completed": [0]})
        quality_check_dat = pd.concat([quality_check_dat, row])

In [ ]:
students_with_submission = len(quality_check_dat.loc[quality_check_dat["submission_found"] == 1])
students_without_submission = len(quality_check_dat.loc[quality_check_dat["submission_found"] == 0])
students_with_submission_but_missing_indicators = len(quality_check_dat.loc[(quality_check_dat["submission_found"] == 1) & ((quality_check_dat["R_completed"] == 0) | (quality_check_dat["Radv_completed"] == 0) | (quality_check_dat["Python_completed"] == 0))])

print(f"{students_with_submission} students with submission")
print(f"{students_without_submission} students without submission")
print(f"{students_with_submission_but_missing_indicators} students with submission but missing indicators")

quality_check_dat

In [ ]:
# students who did not complete R
quality_check_dat[(quality_check_dat["submission_found"] == 1) & (quality_check_dat["R_completed"] == 0)]

In [ ]:
# students who did not complete either Radv or python
quality_check_dat[(quality_check_dat["submission_found"] == 1) & (quality_check_dat["Radv_completed"] == 0) & (quality_check_dat["Python_completed"] == 0)]

In [ ]:
# students who did not complete Radv 
quality_check_dat[(quality_check_dat["submission_found"] == 1) & (quality_check_dat["Radv_completed"] == 0)]

In [ ]:
# students who did not complete Python
quality_check_dat[(quality_check_dat["submission_found"] == 1) & (quality_check_dat["Python_completed"] == 0)]

## LLM Prompting

In [13]:
import json
from markdown_pdf import MarkdownPdf, Section
from openai import OpenAI
from openai._types import NOT_GIVEN
import re
from typing import Tuple

from scripts.utils import load_jsonified_resources


In [ ]:
import os

In [16]:
# LLM settings
UVA_OPENAI_API_KEY = os.getenv("UVA_OPENAI_API_KEY")
UVA_OPENAI_BASE_URL = os.getenv("UVA_OPENAI_BASE_URL")
MODEL = "gpt4o" # this is the correct name for UVA API
openai_client = OpenAI(api_key=UVA_OPENAI_API_KEY, base_url=UVA_OPENAI_BASE_URL)

In [17]:
def load_latest_student_submission(student_id: int, week: int, submissions_path: str = "submissions"):
    """Loads a jsonified submission for a given student and week."""
    file_list = create_file_list(f"{submissions_path}/stu-{student_id}/week-{week}", ["submission.json"])
    if len(file_list) == 0:
        return False, None, None
    file_list = sorted(file_list, key=lambda x: int(re.search(r'try-(\d+)', x).group(1)))
    submission = json.load(open(file_list[-1])) 
    submission = {k: "\n".join(v) for k, v in submission.items()}
    attempt = int(re.search(r'try-(\d+)', file_list[-1]).group(1))
    return True, submission, attempt

def compare_dict_keys(dicts: list) -> Tuple[bool, set]:
    """Checks if the keys of a list of dictionaries are the same."""
    all_keys = {frozenset(d.keys()) for d in dicts}
    if len(all_keys) > 1:
        print("WARNING! Keys are not the same")
        return False, all_keys
    return True, all_keys

In [18]:
def start_report_with_header(report_path: str, model: str, grading_temperature: float, feedback_temperature: float, n_choices_grading: int, n_choices_feedback: int, student_id: int, week: int):
    template = """# LLM Prompt Report
- model: {model}
- grading_temperature: {grading_temperature}
- feedback_temperature: {feedback_temperature}
- n_choices_grading: {n_choices_grading}
- n_choices_feedback: {n_choices_feedback}
- student_id: {student_id}
- week: {week}\n\n"""
    text = template.format(model=model, 
                           grading_temperature=grading_temperature, 
                           feedback_temperature=feedback_temperature, 
                           n_choices_grading=n_choices_grading, 
                           n_choices_feedback=n_choices_feedback, 
                           student_id=student_id, week=week)
    os.makedirs(os.path.dirname(report_path), exist_ok=True)
    with open(report_path, "w") as f:
        f.write(text)

def add_messages_to_report(report_path: str, messages: list, header: str = "### Messages\n"):
    """Adds messages to a report."""
    template = """<blockquote>
<strong>{role}</strong>

\t{content}
</blockquote>\n\n"""

    text = header
    for message in messages:
        text += template.format(role=message["role"], content="\n\t".join(message["content"].split("\n")))
    with open(report_path, "a") as f:
        f.write(text)

def add_text_to_report(report_path: str, text: str):
    with open(report_path, "a") as f:
        f.write(text)
    
def remove_pseudo_html(text: str, tag: str):
    return text.replace(f"<{tag}>", "").replace(f"</{tag}>", "")

def get_majority_voted_points(choices: list):
    points = [float(re.compile(r"<points>(\d+(?:\.\d+)?)</points>").search(choice.message.content).group(1)) for choice in choices]
    return max(set(points), key=points.count)

def get_majority_voted_points_and_explanation(choices: list) -> Tuple[float, str]:
    choice_dict = {}
    points_list = []

    # create a dictionary with points and explanation
    for i, choice in enumerate(choices):
        points = float(re.compile(r"<points>(\d+(?:\.\d+)?)</points>").search(choice.message.content).group(1))
        explanation = re.compile(r"<explanation>(.*)</explanation>", re.DOTALL).search(choice.message.content).group(1)
        choice_dict[i] = {"points": points, "explanation": explanation}
        points_list.append(points)


    # find majority voted points
    majority_voted_points = max(set(points_list), key=points_list.count)

    # select the first explanation with the majority voted points
    explanation = [v["explanation"] for k, v in choice_dict.items() if v["points"] == majority_voted_points][0]
    return majority_voted_points, explanation

def get_total_points_by_question_group(grading_dict, questions, groups = [r"#R\d+", r"#Radv\d+", r"#Python\d+"]) -> dict:
    total_points = {}
    for group in groups:
        possible_points = len([k for k in questions.keys() if re.match(group, k)])
        achieved_points = sum([v["points"] for k, v in grading_dict.items() if re.match(group, k)])
        total_points[group.replace(r"\d+", "")] = {"possible_points": possible_points, "achieved_points": achieved_points}
    return total_points

def get_preliminary_grade(total_points, calculations = {"Radv": {"#R": 0.8, "#Radv": 0.2}, "Python": {"#R": 0.8, "#Python": 0.2}}, max_grade = 10) -> float:
    grade = 0
    print(total_points)
    percent = {k: v["achieved_points"]/v["possible_points"] for k, v in total_points.items()}
    grade = {k: 0 for k in calculations.keys()}
    for group, calc in calculations.items():
        for k, v in calc.items():
            grade[group] += percent[k] * v
        grade[group] *= max_grade
    # find which grade is higher and return that
    return round(max(grade.values()), 3)


In [19]:
from scripts.llm_utils import create_openai_message
from datetime import datetime


def calculate_student_grade(dat, r_weight = 8, adv_py_weight = 2):
    r_points = dat[dat['question'].str.startswith(r'#R') & ~dat['question'].str.startswith(r'#Radv')].points
    r_adv_points = dat[dat['question'].str.startswith('#Radv')].points
    py_points = dat[dat['question'].str.startswith('#Python')].points
    weighted_r = sum(r_points) / len(r_points) * r_weight
    weighted_r_adv = sum(r_adv_points) / len(r_adv_points) * adv_py_weight
    weighted_py = sum(py_points) / len(py_points) * adv_py_weight
    if sum(r_adv_points) > 0:
        return round(weighted_r, 2), round(weighted_r_adv, 2), "You were graded based on Radv."
    return round(weighted_r, 2), round(weighted_py, 2), "You were graded based on Python."

In [20]:
OUT_PATH_PREPEND = "NEW_PROMPTS_TEST/"
OUT_PATH_PREPEND = ""

In [ ]:
# load jsonified resources
jsonified_resources = load_jsonified_resources(WEEK_NUMBER) # TODO: Check all keys and log if it is incorrect

# get assignment
course = canvas.get_course(COURSE_ID)
assignment = course.get_assignment(ASSIGNMENT_ID)
quiz = course.get_quiz(QUIZ_ID)
quiz_submissions = [quiz_submission for quiz_submission in quiz.get_submissions()]
students_df = pd.read_csv(f"{SUBMISSIONS_PATH}/students.csv")

# For each student
for student_id in tqdm(student_ids):
    print(f"Processing student {student_id}")

    # Load jsonified submission
    submission_found, submission, attempt = load_latest_student_submission(student_id, WEEK_NUMBER)
    if not submission_found:
        print(f"Student {student_id} has no submission for week {WEEK_NUMBER}") # TODO: Add log
        continue

    # Validate that all keys are the same
    valid_keys, all_keys = compare_dict_keys([submission] + list(jsonified_resources.values()))
    if not valid_keys:
        print(f"Student {student_id} has different keys in submission and resources for week {WEEK_NUMBER}")
        

    # Prepare some dicts
    grading_dict = {}
    feedback_qw_dict = {}

    # Initialize LLM report
    llm_report_out_path = LLM_COMPLETION_REPORT_TEMPLATE.format(
        student_id=student_id,
        week=WEEK_NUMBER,
        assignment_id=ASSIGNMENT_ID,
        attempt=attempt
    )

    llm_report_out_path = OUT_PATH_PREPEND + llm_report_out_path

    # Skip if the report exists
    if os.path.exists(llm_report_out_path):
        print(f"Student {student_id} already has a report for their most recent attempt of week {WEEK_NUMBER}")
        continue

    start_report_with_header(llm_report_out_path, 
                             MODEL, 
                             GRADING_TEMPERATURE, 
                             FEEDBACK_TEMPERATURE, 
                             N_CHOICES_GRADING, 
                             N_CHOICES_FEEDBACK, 
                             student_id, 
                             WEEK_NUMBER)

    # Loop through questions
    i = 1
    for key in jsonified_resources["rubrics"].keys():
        if i > 200: # for testing
            break
        i += 1
        # Add to report
        add_text_to_report(llm_report_out_path, f"## Question {key}\n")

        # Get question, solution, rubric, goal, and answer
        question = jsonified_resources["questions"][key]
        solution = jsonified_resources["solutions"][key]
        rubric = jsonified_resources["rubrics"][key] + "\n" + "\n".join(GLOBAL_RUBRICS)
        goal = jsonified_resources["goals"][key]

        if key not in submission:
            answer = "!Student did not attempt the task!"
        else:
            answer = submission[key]


        # Prepare the grading prompt
        grading_content_prompt = PROMPTS["grading"]["user_prompt"].format(
            task=question, 
            solution=solution, 
            rubric=rubric, 
            answer=answer)
        messages = create_openai_message("system", PROMPTS["grading"]["system_prompt"])
        messages += create_openai_message("user", grading_content_prompt)

        # Add grading prompt to report
        add_text_to_report(llm_report_out_path, f"<details>\n\t<summary>Grading</summary>\n\n")
        add_messages_to_report(llm_report_out_path, messages, header="#### Prompts\n")

        # Get LLM grading
        completion = openai_client.chat.completions.create(model=MODEL, messages=messages, n=N_CHOICES_GRADING, temperature=GRADING_TEMPERATURE)
        
        # Add to grading dict
        majority_points, majority_explanation = get_majority_voted_points_and_explanation(completion.choices) 
        grading_dict[key] = {"points": majority_points, "explanation": majority_explanation}

        # Add completion to report 
        completion_messages = [{"role": choice.message.role, "content": choice.message.content} for choice in completion.choices]
        add_messages_to_report(llm_report_out_path, completion_messages, header="#### Completion Choices\n")
        add_text_to_report(llm_report_out_path, f"\n\n</details>\n\n")

        # Prepare the feedback prompt
        feedback_qw_content_prompt = PROMPTS["feedback_questionwise"]["user_prompt"].format(
            task=question,
            goal=goal,
            answer=answer,
            r="{r}")
        messages = create_openai_message("system", PROMPTS["feedback_questionwise"]["system_prompt"])
        messages += create_openai_message("user", feedback_qw_content_prompt)

        # Add feedback prompt to report
        add_text_to_report(llm_report_out_path, f"<details>\n\t<summary>Feedback</summary>\n\n")
        add_messages_to_report(llm_report_out_path, messages, header="#### Prompts\n")

        # Get LLM feedback
        completion = openai_client.chat.completions.create(model=MODEL, messages=messages, n=N_CHOICES_FEEDBACK, temperature=FEEDBACK_TEMPERATURE)

        # Add to feedback dict
        feedback_qw_dict[key] = completion.choices[0].message.content # TODO: If we use multiple n this needs to change

        # Add completion to report
        completion_messages = [{"role": choice.message.role, "content": choice.message.content} for choice in completion.choices]
        add_messages_to_report(llm_report_out_path, completion_messages, header="#### Completion Choices\n")
        add_text_to_report(llm_report_out_path, f"\n\n</details>\n\n")


    # Feedback Summary
    add_text_to_report(llm_report_out_path, f"## Feedback Summary\n")

    # Prepare the feedback summary prompt
    feedback_sum_content_prompt = PROMPTS["feedback_summary"]["user_prompt"].format(
        feedback = "\n\n".join([f"<question>{k}</question>\n{v}" for k, v in feedback_qw_dict.items()]))
    messages = create_openai_message("system", PROMPTS["feedback_summary"]["system_prompt"])
    messages += create_openai_message("user", feedback_sum_content_prompt)

    # Add feedback summary prompt to report
    add_text_to_report(llm_report_out_path, f"<details>\n\t<summary>Feedback</summary>\n\n")
    add_messages_to_report(llm_report_out_path, messages, header="#### Prompts\n")

    # Get LLM feedback
    completion = openai_client.chat.completions.create(model=MODEL, messages=messages, n=N_CHOICES_FEEDBACK, temperature=FEEDBACK_TEMPERATURE)

    # Add completion to report
    completion_messages = [{"role": choice.message.role, "content": choice.message.content} for choice in completion.choices]
    add_messages_to_report(llm_report_out_path, completion_messages, header="#### Completion Choices\n")
    add_text_to_report(llm_report_out_path, f"\n\n</details>\n\n")

    # Create feedback report for student
    pdf = MarkdownPdf()

    # Header, summary, preliminary grade, coding challenge
    section = f"# Feedback Assignment {WEEK_NUMBER}\n\n"
    section += "## Summary\n"
    section += re.compile(r"<summary>(.*)</summary>", re.DOTALL).search(completion.choices[0].message.content).group(1)
    section += "\n\n"
    # section += f"**Preliminary grade**: {calculate_preliminary_grade(grading_dict, jsonified_resources['questions'])}/10\n\n"
    section += "## Coding Challenge\n"
    section += "We invite you to work on the following personalized coding challenge and submit your result on Canvas. Dont worry about being perfect, this is ungraded and just for your practice.\n\n"
    section += re.compile(r"<coding-challenge>(.*)</?coding-challenge>", re.DOTALL).search(completion.choices[0].message.content).group(1)
    pdf.add_section(Section(section))

    # Questionwise feedback
    section = ""
    for key, value in feedback_qw_dict.items():
        section += f"## {key}\n"
        remove_idx = value.find("</my-thoughts>") + len("</my-thoughts>")
        if remove_idx > len("</my-thoughts>"):
            value = value[remove_idx:]
        remove_idx = value.find("<my-thoughts>") + len("<my-thoughts>")
        if remove_idx > len("<my-thoughts>"):
            value = value[remove_idx:]
        section += value
        section += "\n\n--\n\n"

    pdf.add_section(Section(section))

    # Save pdf
    student_feedback_report_out_path = LLM_FEEDBACK_REPORT_TEMPLATE.format(
        student_id=student_id,
        week=WEEK_NUMBER,
        assignment_id=ASSIGNMENT_ID,
        attempt=attempt
    )
    student_feedback_report_out_path = OUT_PATH_PREPEND + student_feedback_report_out_path
    pdf.save(student_feedback_report_out_path)

    # Create grading report for student
    pdf = MarkdownPdf()

    # Header, summary, preliminary grade, coding challenge
    section = f"# LLM Grading Assignment {WEEK_NUMBER}\n\n"
    total_points = get_total_points_by_question_group(grading_dict, jsonified_resources["questions"])
    preliminary_grade = get_preliminary_grade(total_points)
    for group, points in total_points.items():
        section += f"**{group}:** {points['achieved_points']}/{points['possible_points']} points\n\n"
    section += f"**Preliminary grade:** {preliminary_grade}/10\n\n"

    
    # Questionwise grading
    for key, value in grading_dict.items():
        # section = ""
        section += f"## {key} ({value['points']}/1)\n"
        section += value["explanation"]
        section += "\n\n"
    pdf.add_section(Section(section))

    # Save pdf
    student_grading_report_out_path = LLM_GRADING_REPORT_TEMPLATE.format(
        student_id=student_id,
        week=WEEK_NUMBER,
        assignment_id=ASSIGNMENT_ID,
        attempt=attempt
    )
    student_grading_report_out_path = OUT_PATH_PREPEND + student_grading_report_out_path
    pdf.save(student_grading_report_out_path)

    if datetime.today() >= datetime.strptime(LOCK_GRADES_DATE, "%Y-%m-%d"):
        print("WARNING GRADSE ARE LOCKED AND NO UPDATES TO CANVAS ARE MADE!")
        continue

    # Create dataframe for grading
    grading_df = pd.DataFrame(grading_dict).T.reset_index().rename(columns={"index": "question"})
    missing_questions = set(jsonified_resources["questions"].keys()) - set(grading_df["question"])
    missing_df = pd.DataFrame({"question": list(missing_questions), "points": 0, "explanation": "Did not attempt"})
    grading_df = pd.concat([grading_df, missing_df], axis=0)

    # calculate grade and create comment
    r_grade, adv_grade, adv_grade_comment = calculate_student_grade(grading_df)
    total_grade = r_grade + adv_grade


    # Get the quiz_submission for this student
    for quiz_submission in quiz_submissions:
        if quiz_submission.user_id == int(student_id):
            break
    
    # Get the canvas submission for this student
    canvas_submission = assignment.get_submission(user = student_id)
    
    # Generate data for updating the score and comments
    if R_QUIZ_QUESTION_ID == ADV_QUIZ_QUESTION_ID:
        data = [{"attempt": canvas_submission.attempt,
                 "questions": {str(R_QUIZ_QUESTION_ID): {"score": total_grade, "comment": adv_grade_comment}}}]
    else:
        data = [{"attempt": canvas_submission.attempt,
                 "questions": {
                     str(R_QUIZ_QUESTION_ID): {"score": r_grade},
                     str(ADV_QUIZ_QUESTION_ID): {"score": adv_grade, "comment": adv_grade_comment}}}]

    # Update the question scores and comments
    quiz_submission.update_score_and_comments(quiz_submissions=data)

    # Post a general comment
    comment = """Be aware that this is a preliminary grade (and comment) based on an automated process.
We provide this to ensure you get a feeling for how last weeks assignment went as fast as possible.

However, the automated system can make mistakes. 
Therefore, we will grade your assignment independently and your final grade may differ."""
    canvas_submission.edit(comment = {"text_comment": comment})

    # Upload the feedback report if student is in group A
    group = students_df.loc[students_df["user_id"] == student_id, f"week-{WEEK_NUMBER}"].values[0]
    if group == "A":
        group_a_comment = """This week you received personalized feedback on your assignment. 

It consists of a summary, an optional coding challenge and questionwise feedback. 

We encourage you to use this feedback to reflect on your assignment and to try working on the optional coding challenge."""
        canvas_submission.edit(comment = {"text_comment": group_a_comment})
        canvas_submission.upload_comment(file=student_feedback_report_out_path)


